In [2]:
import numpy as np
import os
import random
import wandb
import torch
import logging
from arguments import parser
from torch.utils.data import DataLoader

from train import refinement_run
from datasets import create_dataset
from log import setup_default_logging

from accelerate import Accelerator
from omegaconf import OmegaConf

from torch.distributed import get_rank
torch.autograd.set_detect_anomaly(True)

default_setting = './configs/benchmark/patchcore.yaml'
cfg = parser(jupyter=True, default_setting = default_setting)



accelerator = Accelerator(
    mixed_precision             = cfg.TRAIN.mixed_precision
)

# load dataset
trainset, validset, testset = create_dataset(
    dataset_name  = cfg.DATASET.dataset_name,
    method        = cfg.MODEL.method,
    datadir       = cfg.DATASET.datadir,
    class_name    = cfg.DATASET.class_name,
    img_size      = cfg.DATASET.img_size,
    mean          = cfg.DATASET.mean,
    std           = cfg.DATASET.std,
    aug_info      = cfg.DATASET.aug_info,
    anomaly_ratio = cfg.DATASET.anomaly_ratio,
    **cfg.DATASET.get('params',{})
)

# make save directory
savedir = os.path.join(
                            cfg.DEFAULT.savedir,
                            cfg.DATASET.dataset_name,
                            cfg.DATASET.class_name
                        )

exp_name         = cfg.DEFAULT.exp_name

method            = cfg.MODEL.method
model_name        = cfg.MODEL.model_name
model_params      = cfg.MODEL.get('params',{})

trainset         = trainset
validset         = validset
testset          = testset

batch_size       = cfg.DATASET.batch_size
test_batch_size  = cfg.DATASET.test_batch_size
num_workers      = cfg.DATASET.num_workers

opt_name         = cfg.OPTIMIZER.opt_name
lr               = cfg.OPTIMIZER.lr
opt_params       = cfg.OPTIMIZER.get('params',{})

epochs           = cfg.TRAIN.epochs
log_interval     = cfg.TRAIN.log_interval
use_wandb        = cfg.TRAIN.wandb.use

savedir          = savedir
seed             = cfg.DEFAULT.seed
accelerator      = accelerator
cfg              = cfg

# model = __import__('models').__dict__[method](
#     model_name = model_name,
#     **model_params
#     )    

# define train dataloader
trainloader = DataLoader(
    dataset     = trainset,
    batch_size  = batch_size,
    num_workers = num_workers
)

validloader = DataLoader(
    dataset     = validset,
    batch_size  = test_batch_size,
    num_workers = num_workers
)

# define test dataloader
testloader = DataLoader(
    dataset     = testset,
    batch_size  = test_batch_size,
    shuffle     = False,
    num_workers = num_workers
)

# optimizer = __import__('torch.optim', fromlist='optim').__dict__[opt_name](model.parameters(), lr=lr, **opt_params)

# scheduler = None 

# model = __import__('models').__dict__[method](
#         model_name = model_name,
#         **model_params
#         )   

# model, optimizer, trainloader, testloader, scheduler = accelerator.prepare(
#             model, optimizer, trainloader, testloader, scheduler
#         )


DEFAULT:
  seed: 0
  savedir: ./results
  exp_name: coreset_ratio-coreset_ratio_0.1-anomaly_ratio_0
DATASET:
  datadir: ../Data
  batch_size: 32
  test_batch_size: 128
  num_workers: 8
  dataset_name: MVTecAD
  aug_info:
  - RandomResizedCrop
  anomaly_ratio: 0
  class_name: bottle
  img_size: 224
  params:
    normal_ratio: 1
  mean:
  - 0.485
  - 0.456
  - 0.406
  std:
  - 0.229
  - 0.224
  - 0.225
OPTIMIZER:
  opt_name: SGD
  lr: 0
TRAIN:
  epochs: 1
  grad_accum_steps: 1
  mixed_precision: 'no'
  log_interval: 5
  wandb:
    use: false
    project_name: UAADF
MODEL:
  method: PatchCore
  model_name: wide_resnet50_2
  params:
    input_size:
    - 224
    - 224
    layers:
    - layer2
    - layer3
    num_neighbors: 9
    coreset_sampling_ratio: 0.1
REFINEMENT:
  nb_round: 0



In [5]:
dataloader = testloader
score_list = [] 
for idx, (images, labels, gts) in enumerate(dataloader):
    [score_map, score] = model.get_score_map(images.to(device))
    score_list.append(score.detach())
    break 